In [1]:
from pymatgen.ext.matproj import MPRester
from pymatgen.entries.computed_entries import ComputedEntry, Composition, ComputedStructureEntry
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDEntry, PDPlotter

/private/home/anuroops/anaconda3/envs/ocp-20221025/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
api_key = "dH5V5JF7Hsz5nEeZ5H"
mprester = MPRester(api_key=api_key)

/private/home/anuroops/anaconda3/envs/ocp-20221025/lib/python3.9/site-packages/pymatgen/ext/matproj.py:182: UserWarning: You are using the legacy MPRester. This version of the MPRester will no longer be updated. To access the latest data with the new MPRester, obtain a new API key from https://materialsproject.org/api and consult the docs at https://docs.materialsproject.org/ for more information.
  warnings.warn(


In [4]:
mpid = "mp-1143"
data = mprester.query(criteria={"task_id": mpid}, properties=["structure", "final_energy", "e_above_hull", "formation_energy_per_atom", "energy", "energy_per_atom", "uncorrected_energy_per_atom"])[0]
# data = mprester.query(criteria={"task_id": "mp-1245173"}, properties=["structure", "final_energy", "e_above_hull", "formation_energy_per_atom", "energy"])[0]
structure = data["structure"]
data

{'structure': Structure Summary
 Lattice
     abc : 5.177955250310783 5.177955250310783 5.17795522336617
  angles : 55.28961178917173 55.28961178917173 55.28961598024499
  volume : 87.42003699644115
       A : 4.586845 -2.402514 0.0
       B : 4.586845 2.402514 0.0
       C : 3.328448 0.0 3.966441
     pbc : True True True
 PeriodicSite: Al (1.8491, 0.0000, 0.5867) [0.1479, 0.1479, 0.1479]
 PeriodicSite: Al (4.4020, 0.0000, 1.3966) [0.3521, 0.3521, 0.3521]
 PeriodicSite: Al (8.1002, 0.0000, 2.5699) [0.6479, 0.6479, 0.6479]
 PeriodicSite: Al (10.6530, 0.0000, 3.3798) [0.8521, 0.8521, 0.8521]
 PeriodicSite: O (7.7124, 0.9315, 0.9916) [0.5561, 0.9439, 0.2500]
 PeriodicSite: O (5.6629, -0.7355, 0.2227) [0.7500, 0.4439, 0.0561]
 PeriodicSite: O (6.8392, 0.7355, 3.7437) [0.2500, 0.5561, 0.9439]
 PeriodicSite: O (7.3271, -1.6670, 2.2059) [0.9439, 0.2500, 0.5561]
 PeriodicSite: O (5.1750, 1.6670, 1.7605) [0.0561, 0.7500, 0.4439]
 PeriodicSite: O (4.7898, -0.9315, 2.9748) [0.4439, 0.0561, 0.750

In [5]:
elements = list(elt.name for elt in structure.composition.elements)
print(elements)
entries = mprester.get_entries_in_chemsys(elements)
print(len(entries))

['Al', 'O']
130


In [7]:
ce = ComputedStructureEntry(structure, energy=data["energy"])
pd = PhaseDiagram(entries)
print(pd)
pd.get_e_above_hull(ce)

Al-O phase diagram
3 stable phases: 
Al2O3, O2, Al


0.41219999999999946

In [8]:
entry_dict = {e.entry_id: e for e in entries}
ce2 = entry_dict[mpid]
pd.get_e_above_hull(ce2)

0.0

In [12]:
# pd.get_decomp_and_e_above_hull(ce)[0], pd.get_decomp_and_e_above_hull(ce2)[0]
he, he2 = pd.get_decomp_and_hull_energy_per_atom(ce.composition)[1], pd.get_decomp_and_hull_energy_per_atom(ce2.composition)[1]
he, he2

(-7.893557943, -7.893557943)

In [15]:
ce.correction, ce2.correction

(0.0, -4.122)

In [18]:
ce2.energy_adjustments

[CompositionEnergyAdjustment:
   Name: MP2020 anion correction (oxide)
   Value: -4.122 eV
   Uncertainty: 0.012 eV
   Description: Composition-based energy adjustment (-0.687 eV/atom x 6.0 atoms)
   Generated by: MaterialsProject2020Compatibility]

In [9]:
plotter = PDPlotter(pd)
plotter.show()